In [1]:
#SHANE THOMAS
#CS3650
#FINAL PROJECT - QUANTUM PROGRAM
from random import random
import math
from qiskit import *
%matplotlib inline
from qiskit.tools.visualization import plot_histogram
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

# Quantum Methods

In [2]:
def initQDice():
  qc = QuantumCircuit(5,5)
  qc.h(0)
  qc.h(1)
  qc.h(2)
  qc.h(3)
  qc.h(4)
  qc.measure([0,1,2,3,4], [0,1,2,3,4])
  #simulator = Aer.get_backend('qasm_simulator')
  provider = IBMQ.get_provider('ibm-q')
  qcomp = provider.get_backend('ibmq_quito')
  shots = 1024
  job = execute(qc, backend = qcomp, shots=shots, memory=True)
  result = job.result()
  memory = result.get_memory()
  qDiceRolls = []
  for x in range(0, shots):
    num = int(memory[x], 2)
    qDiceRolls.append(num)
  return qDiceRolls

masterQDice = initQDice()

In [3]:
def qTeleportationDiceKeep():
    global holdD1
    global holdD2
    global holdD3
    global holdD4
    global holdD5
    
    qr = QuantumRegister(5)
    cr = ClassicalRegister(5)
    circuit = QuantumCircuit(qr,cr)

    circuit.x(qr[0])
    circuit.h(qr[4])

    if (holdD1):
      circuit.cx(qr[4],qr[1])
    if (holdD2):
      circuit.cx(qr[4],qr[2])
    if (holdD3):
      circuit.cx(qr[4],qr[3])

    circuit.cx(qr[0],qr[4])
    circuit.h(qr[0])
    circuit.measure(qr[0],cr[0])
    circuit.measure(qr[4],cr[4])

    if (holdD1):
      circuit.cx(qr[4],qr[1])
      circuit.cz(qr[0],qr[1])
    if (holdD2):
      circuit.cx(qr[4],qr[2])
      circuit.cz(qr[0],qr[2])
    if (holdD3):
      circuit.cx(qr[4],qr[3])
      circuit.cz(qr[0],qr[3])

    circuit.measure(qr[1],cr[1])
    circuit.measure(qr[2],cr[2])
    circuit.measure(qr[3],cr[3])
    simulator = Aer.get_backend('qasm_simulator')
    provider = IBMQ.get_provider('ibm-q')
    qcomp = provider.get_backend('ibmq_quito')
    job = execute(circuit, backend = qcomp, shots = 1)
    from qiskit.tools.monitor import job_monitor
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    
    #convert 
    crDict = [(k[::-1],v) for k,v in counts.items()]
    crDict.sort(key = lambda x: x[1], reverse=True)
    crStr = str(crDict)
    output = []
    for i in range(3):
        output.append(crStr[i+4])
    #print (output)

    #################################
    qr = QuantumRegister(5)
    cr = ClassicalRegister(5)
    circuit = QuantumCircuit(qr,cr)

    circuit.x(qr[0])
    circuit.h(qr[4])

    if (holdD4):
      circuit.cx(qr[4],qr[1])
    if (holdD5):
      circuit.cx(qr[4],qr[2])

    circuit.cx(qr[0],qr[4])
    circuit.h(qr[0])
    circuit.measure(qr[0],cr[0])
    circuit.measure(qr[4],cr[4])
    
    if (holdD4):
      circuit.cx(qr[4],qr[1])
      circuit.cz(qr[0],qr[1])
    if (holdD5):
      circuit.cx(qr[4],qr[2])
      circuit.cz(qr[0],qr[2])

    circuit.measure(qr[1],cr[1])
    circuit.measure(qr[2],cr[2])
    circuit.measure(qr[3],cr[3])
    simulator = Aer.get_backend('qasm_simulator')
    qcomp = provider.get_backend('ibmq_quito')
    job = execute(circuit, backend = qcomp, shots = 1)
    from qiskit.tools.monitor import job_monitor
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    
    #convert 
    crDict = [(k[::-1],v) for k,v in counts.items()]
    crDict.sort(key = lambda x: x[1], reverse=True)
    crStr = str(crDict)
    for i in range(2):
        output.append(crStr[i+4])
    #print (output)
    return output

# Methods

In [4]:
#METHODS

def qDiceRoll(allDice):
  idx = math.floor(random() * len(allDice))
  roll = (allDice[idx] + 1)/32
  rollNum = math.ceil(roll * 6)
  return rollNum
################################################################################################
def threeRow(dice):
  score = 0
  for i in dice:
    if dice.count(i) > 2:
      score = dice.count(i) * i
      return score
    else:
      return score
################################################################################################
def fourRow(dice):
  score = 0
  for i in dice:
    if dice.count(i) > 3:
      score = dice.count(i) * i
      return score
    else:
      return score
################################################################################################
def yahtzee(dice):
  score = 0
  for i in dice:
    if dice.count(i) == 5:
      score = 50
      return score
    else:
      return score
################################################################################################
def fullHouse(dice):
  score = 0
  pair = 0
  for i in dice:
    if dice.count(i) == 3:
      pair = 3
    elif dice.count(i) == 2:
      pair = 2
    else:
      return score
  for i in dice:
    if (dice.count(i) == 2 & pair == 3) | (dice.count(i) == 3 & pair == 2):
      score = 25
      return score
    else:
      return score
################################################################################################
def sStraight(dice):
  score = 0
  dice = str(sorted(dice))
  if ("1, 2, 3, 4" in dice) | ("2, 3, 4, 5" in dice) | ("3, 4, 5, 6" in dice):
    score = 30
    return score
  else:
    return score
################################################################################################
def lStraight(dice):
  score = 0
  dice = str(sorted(dice))
  if ("1, 2, 3, 4, 5" in dice) | ("2, 3, 4, 5, 6" in dice):
    score = 40
    return score
  else:
    return score
################################################################################################
def calcScore(dice):
  score = 0
  score = yahtzee(dice)
  if score != 0:
    return score
  score = lStraight(dice)
  if score != 0:
    return score
  score = sStraight(dice)
  if score != 0:
    return score
  score = fullHouse(dice)
  if score != 0:
    return score
  score = fourRow(dice)
  if score != 0:
    return score
  score = threeRow(dice)
  return score
################################################################################################
def intro():
  print ("Q Y A H T Z E E ! ! !\n\n")
  print ("Welcome, Player! This game is similar to Yahtzee, with some quantum mechanics involved!\n\n")
  print ("RULES:\n")
  print ("- You have 3 opportunities to roll the qDice (quantum dice), trying to get the highest score in ONE round\n")
  print ("- You may keep none, some, or all of your dice in-between rounds (The computer will roll every round with ONE exception)\n")
  print ("- IF you decide to keep a die in-between rounds, you will also force the computer to keep their respective die as well (i.e. You keep die 1, they keep die 1, etc.)\n\n")
  print ("SCORING:\n")
  print ("Three of a Kind: (3-18 points) Having three of the same dice\n")
  print ("Four of a Kind: (4-24 points) Having four of the same dice\n")
  print ("Full House: (25 points)  Having three of the same dice & having two of a different pair of dice\n")
  print ("Small Straight: (30 points) Having four numbers in a sequence\n")
  print ("Large Straight: (40 points) Having five numbers in a sequence\n")
  print ("QYAHTZEE: (50 points) Having five of the same dice\n")
  input("\nPress ENTER to stat the game!")
  return
################################################################################################
def firstRoll(dice):
  global player
  if (player):
    input("\nPress ENTER to roll the qDice!")
    print("...\nYOU ROLLED:\n")
  else:
    print("\nThe computer is rolling the qDice!\n...\nCOMPUTER ROLLED:\n")
  return dice
################################################################################################
def roll(diceK, diceR):
  global player
  global holdD1
  global holdD2
  global holdD3
  global holdD4
  global holdD5
  rolledDice = newDice(len(diceR))
  allDice = []
  allDice = diceK + rolledDice

  if (player):
    holdD1 = False
    holdD2 = False
    holdD3 = False
    holdD4 = False
    holdD5 = False
    input("\nPress ENTER to roll the qDice!")
    print("...\nYOU ROLLED:\n")
  else:
    print("\nThe computer is rolling the qDice!\n...\nCOMPUTER ROLLED:\n")
  return allDice
################################################################################################
def newDice(num):
  dice = []
  global masterQDice
  for i in range(num):
    #dice.append(1)
    dice.append(qDiceRoll(masterQDice))
  return dice
################################################################################################
def lastRoll(dice):
    global player
    
    if (player):
        player = False
        done = False
        print("\nDIE #1: " + str(dice[0]))
        print("\nDIE #2: " + str(dice[1]))
        print("\nDIE #3: " + str(dice[2]))
        print("\nDIE #4: " + str(dice[3]))
        print("\nDIE #5: " + str(dice[4]))
    
    else:
        player = True
        print("\nDIE #1: " + str(dice[0]))
        print("\nDIE #2: " + str(dice[1]))
        print("\nDIE #3: " + str(dice[2]))
        print("\nDIE #4: " + str(dice[3]))
        print("\nDIE #5: " + str(dice[4]))
    
    return
################################################################################################
def keep(dice):
  global fRoll
  global player
  global holdD1
  global holdD2
  global holdD3
  global holdD4
  global holdD5
  kDice = []
  if (player):
    player = False
    done = False
    print("\nDIE #1: " + str(dice[0]))
    print("\nDIE #2: " + str(dice[1]))
    print("\nDIE #3: " + str(dice[2]))
    print("\nDIE #4: " + str(dice[3]))
    print("\nDIE #5: " + str(dice[4]))
    while (True):
      k = 0
      print ("\n\nDice to keep: ")
      if holdD1:
        print("DIE #1  ")
        if done:
          kDice.append(dice[0])
      if holdD2:
        print("DIE #2  ")
        if done:
          kDice.append(dice[1])
      if holdD3:
        print("DIE #3  ")
        if done:
          kDice.append(dice[2])
      if holdD4:
        print("DIE #4  ")
        if done:
          kDice.append(dice[3])
      if holdD5:
        print("DIE #5")
        if done:
          kDice.append(dice[4])
      if not (holdD1) and not (holdD2) and not (holdD3) and not (holdD4) and not (holdD5):
          print("NONE\n")
      if done:
        break
      k = int(input("\nPlease select which dice to keep/remove (enter 1-5, 0 if finished): "))
      if k == 1:
        if not holdD1:
          holdD1 = True
        else:
          holdD1 = False
      if k == 2:
        if not holdD2:
          holdD2 = True
        else:
          holdD2 = False
      if k == 3:
        if not holdD3:
          holdD3 = True
        else:
          holdD3 = False
      if k == 4:
        if not holdD4:
          holdD4 = True
        else:
          holdD4 = False
      if k == 5:
        if not holdD5:
          holdD5 = True
        else:
          holdD5 = False
      if k == 0:
        done = True
      
  else:
    player = True
    print("\nDIE #1: " + str(dice[0]))
    print("\nDIE #2: " + str(dice[1]))
    print("\nDIE #3: " + str(dice[2]))
    print("\nDIE #4: " + str(dice[3]))
    print("\nDIE #5: " + str(dice[4]))
    hold = []
    hold = qTeleportationDiceKeep()
    print("\nThe computer has kept the following dice: ")
    if int(hold[0]) == 1:
        print("DIE #1  ")
        kDice.append(dice[0])
    if int(hold[1]) == 1:
        print("DIE #2  ")
        kDice.append(dice[1])
    if int(hold[2]) == 1:
        print("DIE #3  ")
        kDice.append(dice[2])
    if int(hold[3]) == 1:
        print("DIE #4  ")
        kDice.append(dice[3])
    if int(hold[4]) == 1:
        print("DIE #5")
        kDice.append(dice[4])
    if not (int(hold[0]) == 1) and not (int(hold[1]) == 1) and not (int(hold[2]) == 1) and not (int(hold[3]) == 1) and not (int(hold[4]) == 1):
        print("NONE\n")

  return kDice
#####################################################################################################################
def noKeep(dice):
  global holdD1
  global holdD2
  global holdD3
  global holdD4
  global holdD5
  nkDice = []
  if not holdD1:
    nkDice.append(dice[0])
  if not holdD2:
    nkDice.append(dice[1])
  if not holdD3:
    nkDice.append(dice[2])
  if not holdD4:
    nkDice.append(dice[3])
  if not holdD5:
    nkDice.append(dice[4])
  return nkDice
#####################################################################################################################
def resetGame():
    global playerScore
    playerScore = 0
    global compScore
    compScore = 0
    global holdD1
    holdD1 = False
    global holdD2
    holdD2 = False
    global holdD3
    holdD3 = False
    global holdD4
    holdD4 = False
    global holdD5
    holdD5 = False
    global player
    player = True
    return

# MAIN()

In [11]:
#MAIN CODE
intro()
playerScore = 0
compScore = 0
holdD1 = False
holdD2 = False
holdD3 = False
holdD4 = False
holdD5 = False
player = True
global masterQDice
while (True):
  resetGame()
  #FIRST ROLL
  print("\n\nFIRST ROLL!")
  pDice = newDice(5)
  cDice = newDice(5)
  pDice = firstRoll(pDice)
  pKeep = keep(pDice)
  pNoKeep = noKeep(pDice)
  cDice = firstRoll(cDice)
  cKeep = keep(cDice)
  cNoKeep = noKeep(cDice)
  while (len(cKeep + cNoKeep) < 5):
    cNoKeep.append(1)
  #SECOND ROLL
  print("\nSECOND ROLL!")
  pDice = roll(pKeep, pNoKeep)
  pKeep = keep(pDice)
  pNoKeep = noKeep(pDice)
  cDice = roll(cKeep, cNoKeep)
  cKeep = keep(cDice)
  cNoKeep = noKeep(cDice)
  while (len(cKeep + cNoKeep) < 5):
    cNoKeep.append(1)
  #THIRD ROLL
  print("\nLAST ROLL!!!")
  pDice = roll(pKeep, pNoKeep)
  lastRoll(pDice)
  cDice = roll(cKeep, cNoKeep)
  lastRoll(cDice)
  #SCORE
  playerScore = calcScore(pDice)
  compScore = calcScore(cDice)
  input("THAT'S THE GAME! Press enter to calculate scores!")
  print ("\n\nPlayer Score: " + str(playerScore))
  print ("Computer Score: " + str(compScore))
  if playerScore > compScore:
    playMore = input("\nCongrats on winning! If you'd like to play again, press ENTER. To quit, press 'q' ")
  elif playerScore == compScore:
    playMore = input("\nIt's a tie! If you'd like to play again, press ENTER. To quit, press 'q' ")
  else:
    playMore = input("\nYou lost! If you'd like to play again, press ENTER. To quit, press 'q' ")
  if playMore == "q":
    break

Q Y A H T Z E E ! ! !


Welcome, Player! This game is similar to Yahtzee, with some quantum mechanics involved!


RULES:

- You have 3 opportunities to roll the qDice (quantum dice), trying to get the highest score in ONE round

- You may keep none, some, or all of your dice in-between rounds (The computer will roll every round with ONE exception)

- IF you decide to keep a die in-between rounds, you will also force the computer to keep their respective die as well (i.e. You keep die 1, they keep die 1, etc.)


SCORING:

Three of a Kind: (3-18 points) Having three of the same dice

Four of a Kind: (4-24 points) Having four of the same dice

Full House: (25 points)  Having three of the same dice & having two of a different pair of dice

Small Straight: (30 points) Having four numbers in a sequence

Large Straight: (40 points) Having five numbers in a sequence

QYAHTZEE: (50 points) Having five of the same dice


Press ENTER to stat the game!


FIRST ROLL!

Press ENTER to roll the qDice!